In [1]:
import pandas as pd
import configparser
import boto3
import json

# STEP 0: Read Configuration Parameters

In [2]:
config = configparser.ConfigParser()
config.read_file(open('dwh.cfg'))

In [3]:
KEY = config.get('AWS','KEY')
SECRET = config.get('AWS', 'SECRET')

In [4]:
IAM_ROLE_ARN = config.get('IAM_ROLE','IAM_ROLE_ARN')

In [5]:
CLUSTER_IDENTIFIER = config.get('REDSHIFT','CLUSTER_IDENTIFIER')
CLUSTER_TYPE = config.get('REDSHIFT','CLUSTER_TYPE')
NUM_NODES = config.get('REDSHIFT','NUM_NODES')
NODE_TYPE = config.get('REDSHIFT','NODE_TYPE')

In [6]:
DB_NAME = config.get('CLUSTER','DB_NAME')
DB_USER = config.get('CLUSTER','DB_USER')
DB_PASSWORD = config.get('CLUSTER','DB_PASSWORD')
DB_PORT = config.get('CLUSTER','DB_PORT')

# STEP 1: Set Up IAM Role

In [7]:
iam = boto3.client(
    'iam',
    region_name = 'us-west-2',
    aws_access_key_id = KEY,
    aws_secret_access_key = SECRET
)

In [8]:
try:
    print('1.1 Creating a new IAM Role')
    dwhRole = iam.create_role(
      Path = '/',
      RoleName = IAM_ROLE_ARN,
      AssumeRolePolicyDocument = json.dumps(
            {'Statement': [{'Action': 'sts:AssumeRole',
                           'Effect': 'Allow',
                           'Principal': {'Service': 'redshift.amazonaws.com'}}],
            'Version': '2012-10-17'}
        ),
      Description='Allows Redshift to call AWS services on your behalf.'
    )
except Exception as e:
    print(e)
    
try:
    print('1.2 Attaching Policy')
    iam.attach_role_policy(RoleName=IAM_ROLE_ARN,
                          PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"
                          )['ResponseMetadata']['HTTPStatusCode']
except Exception as e:
    print(e)
    
try:
    print('1.3 Get the IAM role ARN')
    roleArn = iam.get_role(RoleName=IAM_ROLE_ARN)['Role']['Arn']
except Exception as e:
    print(e)

1.1 Creating a new IAM Role
1.2 Attaching Policy
1.3 Get the IAM role ARN


# STEP 2: Create Redshift Cluster

In [9]:
redshift = boto3.client(
    'redshift',
    region_name = 'us-west-2',
    aws_access_key_id = KEY,
    aws_secret_access_key = SECRET
)

In [10]:
try:
    response = redshift.create_cluster(
        # TODO: add parameters for hardware
        ClusterType=CLUSTER_TYPE,
        NodeType=NODE_TYPE,
        NumberOfNodes=int(NUM_NODES),

        # TODO: add parameters for identifiers & credentials
        DBName=DB_NAME,
        ClusterIdentifier=CLUSTER_IDENTIFIER,
        MasterUsername=DB_USER,
        MasterUserPassword=DB_PASSWORD,
        
        # TODO: add parameter for role (to allow s3 access)
        IamRoles=[roleArn]
    )
except Exception as e:
    print(e)

In [23]:
myClusterProps = redshift.describe_clusters(ClusterIdentifier=CLUSTER_IDENTIFIER)['Clusters'][0]
myClusterProps

ClusterNotFoundFault: An error occurred (ClusterNotFound) when calling the DescribeClusters operation: Cluster sparkifycluster not found.

In [18]:
config.set('CLUSTER','HOST',myClusterProps['Endpoint']['Address'])
config.set('CLUSTER','DWH_IAM_ROLE',myClusterProps['IamRoles'][0]['IamRoleArn'])
with open('dwh.cfg', 'w') as configfile:
    config.write(configfile)

# Step 3: Open an incoming  TCP port to access the cluster ednpoint

In [21]:
ec2 = boto3.resource(
    'ec2',
    region_name = 'us-west-2',
    aws_access_key_id = KEY,
    aws_secret_access_key = SECRET
)

In [23]:
try:
    vpc = ec2.Vpc(id=myClusterProps['VpcId'])
    defaultSg = list(vpc.security_groups.all())[0]
    print(defaultSg)
    
    defaultSg.authorize_ingress(
        GroupName= defaultSg.group_name,  # TODO: fill out
        CidrIp='0.0.0.0/0',  # TODO: fill out
        IpProtocol='TCP',  # TODO: fill out
        FromPort=int(DB_PORT),
        ToPort=int(DB_PORT)
    )
except Exception as e:
    print(e)

ec2.SecurityGroup(id='sg-83268fce')
An error occurred (InvalidPermission.Duplicate) when calling the AuthorizeSecurityGroupIngress operation: the specified rule "peer: 0.0.0.0/0, TCP, from port: 5439, to port: 5439, ALLOW" already exists


# Step 4: Delete Resources

In [19]:
redshift.delete_cluster( ClusterIdentifier=CLUSTER_IDENTIFIER,  SkipFinalClusterSnapshot=True)

{'Cluster': {'ClusterIdentifier': 'sparkifycluster',
  'NodeType': 'dc2.large',
  'ClusterStatus': 'deleting',
  'MasterUsername': 'sparkify_user',
  'DBName': 'sparkify_dwh',
  'Endpoint': {'Address': 'sparkifycluster.cchz83t4uoes.us-west-2.redshift.amazonaws.com',
   'Port': 5439},
  'ClusterCreateTime': datetime.datetime(2019, 6, 12, 12, 56, 28, 496000, tzinfo=tzlocal()),
  'AutomatedSnapshotRetentionPeriod': 1,
  'ClusterSecurityGroups': [],
  'VpcSecurityGroups': [],
  'ClusterParameterGroups': [{'ParameterGroupName': 'default.redshift-1.0',
    'ParameterApplyStatus': 'in-sync'}],
  'ClusterSubnetGroupName': 'default',
  'VpcId': 'vpc-79f05a01',
  'AvailabilityZone': 'us-west-2c',
  'PreferredMaintenanceWindow': 'mon:12:30-mon:13:00',
  'PendingModifiedValues': {},
  'ClusterVersion': '1.0',
  'AllowVersionUpgrade': True,
  'NumberOfNodes': 4,
  'PubliclyAccessible': True,
  'Encrypted': False,
  'Tags': [],
  'EnhancedVpcRouting': False,
  'IamRoles': [{'IamRoleArn': 'arn:aws:ia

In [29]:
iam.detach_role_policy(RoleName=IAM_ROLE_ARN, PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess")
iam.delete_role(RoleName=IAM_ROLE_ARN)

{'ResponseMetadata': {'RequestId': '9794e38e-8d09-11e9-9963-599d8b6500ae',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '9794e38e-8d09-11e9-9963-599d8b6500ae',
   'content-type': 'text/xml',
   'content-length': '200',
   'date': 'Wed, 12 Jun 2019 11:59:54 GMT'},
  'RetryAttempts': 0}}